# Семинар 3: Векторы слов

Другие курсы на ту же тему:
* https://github.com/DanAnastasyev/DeepNLP-Course: Курс Данила Анастасьева, Week 2
* https://www.youtube.com/watch?v=ERibwqs9p38: Stanford CS224n, Lecture 2
* https://github.com/deepmipt/deep-nlp-seminars: Курс DeepMIPT, Seminar 2

In [ ]:
!apt-get update
!apt-get install -y python-setuptools python-pip
!pip install --upgrade pybind11 setuptools

In [ ]:
%%writefile requirements.txt
gensim
pandas
razdel
hnswlib

In [ ]:
!pip install --upgrade -r requirements.txt

### Скачиваем датасет на сегодня

In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz
!gzip -d lenta-ru-news.csv.gz
!head -n 2 lenta-ru-news.csv

### Обрабатываем датасет

In [ ]:
import pandas as pd

dataset = pd.read_csv("lenta-ru-news.csv", sep=',', quotechar='\"', escapechar='\\', encoding='utf-8', header=0)
dataset.head()

In [ ]:
import re
import datetime as dt

def get_date(url):
    dates = re.findall(r"\d\d\d\d\/\d\d\/\d\d", url)
    return next(iter(dates), None)
  
dataset["date"] = dataset["url"].apply(lambda x: dt.datetime.strptime(get_date(x), "%Y/%m/%d"))
dataset = dataset[dataset["date"] > "2017-01-01"]
dataset["text"] = dataset["text"].apply(lambda x: x.replace("\xa0", " "))
dataset["title"] = dataset["title"].apply(lambda x: x.replace("\xa0", " "))
dataset.head()

In [ ]:
train_dataset = dataset[dataset["date"] < "2018-04-01"]
test_dataset = dataset[dataset["date"] > "2018-04-01"]
print(train_dataset.info())
print(test_dataset.info())

# Задачи, которые будем решать
* Семантический поиск по заголовку
* Рубрикация

## Подготовка: разбиение на предложения и токенизация

In [ ]:
from razdel import tokenize, sentenize
from string import punctuation

texts = []
for text in train_dataset["text"]:
    pass
    # Разбейте на предложения
    # Каждое предложение токенизируйте и список токенов положите в texts.
    # Токены приведите к нижнему регистру и избавьтесь от пунктуации.
    
for title in train_dataset["title"]:
    pass
    # Считайте заголовок одним предложением

assert len(texts) == 827217
assert len(texts[0]) > 0
assert texts[0][0].islower()
print(texts[0])

## Коротко о Word2Vec
Обучение:

![embeddings training](https://miro.medium.com/max/1400/0*o2FCVrLKtdcxPQqc.png)
*From [An implementation guide to Word2Vec using NumPy and Google Sheets
](https://towardsdatascience.com/an-implementation-guide-to-word2vec-using-numpy-and-google-sheets-13445eebd281)*

![embeddings relations](https://www.tensorflow.org/images/linear-relationships.png)
*From [Vector Representations of Words, Tensorflow tutorial](https://www.tensorflow.org/tutorials/representation/word2vec)*

Статьи:
* Word2Vec: [Distributed Representations of Words and Phrases
and their Compositionality](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf), Mikolov et al., 2013
* GloVe: [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/pubs/glove.pdf), Pennington, Socher, Manning, 2014
* fastText: [Enriching Word Vectors with Subword Information](https://arxiv.org/pdf/1607.04606.pdf), Bojanowski, Grave, Joulin, Mikolov, 2016

Ссылки:
* Word2Vec и fasttext модели для русского: https://rusvectores.org/ru/
* fasttext для кучи языков: https://fasttext.cc/
* Ещё fasttext модели для русского: http://docs.deeppavlov.ai/en/master/features/pretrained_vectors.html
* Отдельная библиотека для русских векторов: https://github.com/natasha/navec
* Word2Vec для кучи языков, обученная на Вики: https://wikipedia2vec.github.io/wikipedia2vec/pretrained/
* Word2Vec для английского от Гугла: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM
* Огромная Word2Vec модель для русского: https://zenodo.org/record/400631#.Xa4RPN9fjCI

## Тренируем простую модель

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(texts, 
                 size=32,     # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

Полноценная тренировка в следующий раз :)
А теперь немного потестируем нашу модель.

## Тестируем модель

In [ ]:
model.get_vector("сша")

In [ ]:
model.most_similar('сша')

In [ ]:
model.most_similar([model.get_vector('трамп') - model.get_vector('сша') + model.get_vector('россии')])

### Задание: Найдите свою аналогию
Поиграйтесь с моделью и найдите свои аналогии. Можно здесь, можно на rusvectores



## Визуализируем векторы

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    output_notebook()
    
    if isinstance(color, str): 
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({'x' : x, 'y' : y, 'color': color, **kwargs})

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig

In [ ]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]
word_vectors = model.vectors[[model.vocab[word].index for word in words]]

### PCA

Простейший линейный метод сокращения размерностей - __P__rincipial __C__omponent __A__nalysis.

PCA ищет оси, при проекции на которые данные будут иметь наибольший разброс.

![pca](https://i.stack.imgur.com/Q7HIP.gif)
*From [https://stats.stackexchange.com/a/140579](https://stats.stackexchange.com/a/140579)*

В результате, можно взять проекции на несколько первых компонент - и сохранить как можно больше информации, сократив размерность.

Красивые визуализации можно найти [здесь](http://setosa.io/ev/principal-component-analysis/).

In [ ]:
from sklearn.decomposition import PCA
pca_model = PCA(n_components=2)
pca_vectors = pca_model.fit_transform(word_vectors)
pca_vectors = (pca_vectors - pca_vectors.mean(0)) / pca_vectors.std(0)

In [ ]:
draw_vectors(pca_vectors[:, 0], pca_vectors[:, 1], token=words)

Получилось не очень понятно.

### TSNE

Более интересный и сложный (нелинейный) метод для визуализации высокоразмерных пространств - TSNE. Подробно посмотреть на него можно [здесь](https://distill.pub/2016/misread-tsne/) (ещё более красивые картинки!).

### Задание: TSNE
Сделайте то же самое, но с TSNE

In [ ]:
# CODE HERE

In [ ]:
draw_vectors(tsne_vectors[:, 0], tsne_vectors[:, 1], token=words)

## Поиск заголовков

In [ ]:
from razdel import tokenize
import numpy as np

def get_text_embedding(model, phrase):
    embeddings = np.array([model.get_vector(word.text.lower()) if word.text.lower() in model.vocab else np.zeros((model.vector_size,))
                           for word in tokenize(phrase)])
    return np.mean(embeddings, axis=0)
    
get_text_embedding(model, "В Москве нашли")

### Задание: k ближайших заголовков
Напишите функцию для поиска похожих на запрос заголовков


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def find_nearest(model, text_vectors, texts, query, k=10):
    # YOUR CODE HERE
    pass

test_titles = test_dataset["title"].tolist()
title_vectors = np.array([get_text_embedding(model, title) for title in test_titles])

In [ ]:
query = "В Москве нашли"
near_titles = find_nearest(model, title_vectors, test_titles, query)
assert len(near_titles) == 10

near_titles

### HNSW-индекс

Поиск за log(n)

* https://github.com/nmslib/hnswlib
* https://habr.com/ru/company/mailru/blog/338360/
* https://arxiv.org/ftp/arxiv/papers/1603/1603.09320.pdf

In [ ]:
import hnswlib

hnsw = hnswlib.Index(space='cosine', dim=title_vectors.shape[1])
hnsw.init_index(max_elements=title_vectors.shape[0])
hnsw.add_items(title_vectors)

In [ ]:
labels, distances = hnsw.knn_query(get_text_embedding(model, query), k=3)
near_titles = [test_titles[i] for i in labels[0]]
near_titles

## Рубрикация

In [ ]:
target_labels = set(train_dataset["topic"].dropna().tolist())
target_labels -= {"69-я параллель", "Крым", "Культпросвет ", "Оружие", "Бизнес", "Путешествия"}
target_labels = list(target_labels)
print(target_labels)

In [ ]:
pattern = r'(\b{}\b)'.format('|'.join(target_labels))

train_with_topics = train_dataset[train_dataset["topic"].str.contains(pattern, case=False, na=False)]
train_with_topics = train_with_topics.head(20000)

test_with_topics = test_dataset[test_dataset["topic"].str.contains(pattern, case=False, na=False)]

In [ ]:
import numpy as np


y_train = train_with_topics["topic"].apply(lambda x: target_labels.index(x)).to_numpy()
X_train = np.zeros((train_with_topics.shape[0], model.vector_size))
for i, embedding in enumerate(train_with_topics["text"]):
    X_train[i, :] = get_text_embedding(model, embedding)

y_test = test_with_topics["topic"].apply(lambda x: target_labels.index(x)).to_numpy()
X_test = np.zeros((test_with_topics.shape[0], model.vector_size))
for i, embedding in enumerate(test_with_topics["text"]):
    X_test[i, :] = get_text_embedding(model, embedding)

print(X_train.shape)
print(y_train)

In [ ]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train, y_train)

In [ ]:
from sklearn import metrics

y_predicted = clf.predict(X_test)
print(metrics.classification_report(y_test, y_predicted))

### Задание: Больше точности

Увеличить точность на 0.02+ на той же Word2Vec модели (например, используя tf-idf при построении вектора текста)

## Предобученные векторы

### Задание: Модели rusvectores
Используя fastText модели с rusvectores, достигните хотя бы такой же точности рубрикации